In [15]:
import pandas as pd
import random
import re
import json
import os
from talkingtomachines.management.experiment import AItoAIInterviewExperiment
from typing import List

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
TEST_MODE = False
NUM_SESSIONS = 50

# Load and Preprocess Demographic Data

In [ ]:
demographic_col_mapping = {
    'age':'What is your age?',
    'agreeable':'On a scale from 1 to 7, where 1 means "Disagree Strongly" and 7 means "Agree Strongly", how well do the following statement describe your personality: I see myself as someone who is uncritical, peaceful, sympathetic, and warm.',
    'college':'Do you have a college education?', 
    'consecientious':'On a scale from 1 to 7, where 1 means "Disagree Strongly" and 7 means "Agree Strongly", how well do the following statement describe your personality: I see myself as someone who dependable, self-disciplined, organised, and careful.', 
    'democrat':'Are you a democrat?', 
    'emotionally_stable':'On a scale from 1 to 7, where 1 means "Disagree Strongly" and 7 means "Agree Strongly", how well do the following statement describe your personality: I see myself as someone who is calm, not easily upset, calm, emotionally stable.', 
    'extraversion':'On a scale from 1 to 7, where 1 means "Disagree Strongly" and 7 means "Agree Strongly", how well do the following statement describe your personality: I see myself as someone who is extraverted, enthusiastic, unreserved, and loud.', 
    'gender':'What is your gender? 1 refers to Male and 2 refers to Female', 
    'knowledge_counter':'On a scale from 0 to 3, where 0 means "Not at All Knowledgeable" and 3 means "Very Knowledgeable," please rate the following statement: I consider myself knowledgeable about political issues and events.', 
    'open_new_experiences':'On a scale from 1 to 7, where 1 means "Disagree Strongly" and 7 means "Agree Strongly", how well do the following statement describe your personality: I see myself as someone who is open to new experiences, complex, unconventional, and creative.',
    'percent_other_party_exposure':'On a scale from 0 to 100, where 0 means "No Exposure" and 100 means "Extensive Exposure," please indicate the percentage of your political information and interactions that come from sources affiliated with parties other than your own.',
    'political_interest':'On a scale from 1 to 4, where 1 means "Not at All Interested" and 4 means "Very Interested," please rate your level of interest in politics.',
    'regionMidwest':'Do you live in the midwest region of the United States?', 
    'regionSouth':'Do you live in the south region of the United States?',
    'regionWest':'Do you live in the west region of the United States?', 
    'white':'Are you white?', 
    'sevenpoint_ideology':'On a scale from 1 to 7, where 1 means "Very Liberal," 4 means "Moderate," and 7 means "Very Conservative," please rate your political ideology.',  
    'strong_partisan':'Do you strongly align with your current political party and actively support its policies and candidates?', 
    'republican':'Are you a republican?', 
    'faminc_val':'faminc_val',
    'faminc_notsay':'faminc_notsay',
    'index':'ID'
}

demographic_cols = list(demographic_col_mapping.values())
demographics_data = pd.read_csv("reducing_political_polarisation.csv")
demographics_data = demographics_data.reset_index(drop=False).rename(columns=demographic_col_mapping)
demographics_data.head()

In [17]:
# Split into treatment 1 and treatment 2
placebo_demographics_data = demographics_data[demographics_data["treatment_assigned"]==0].reset_index(drop=True)
treatment_demographics_data = demographics_data[demographics_data["treatment_assigned"]==1].reset_index(drop=True)

# For placebo treatment group, split into immigration group and gun control group randomly
placebo_session_ids = list(placebo_demographics_data["chatID_full"].unique())
split_size = int(len(placebo_session_ids) * 0.5)
placebo_guncontrol_session_ids = random.sample(placebo_session_ids, split_size)
placebo_immigration_session_ids = [id for id in placebo_session_ids if id not in placebo_guncontrol_session_ids]

# placebo_guncontrol_session_ids = placebo_guncontrol_session_ids[:NUM_SESSIONS]
# placebo_immigration_session_ids = placebo_immigration_session_ids[:NUM_SESSIONS]

placebo_guncontrol_data = placebo_demographics_data[placebo_demographics_data["chatID_full"].isin(placebo_guncontrol_session_ids)].reset_index(drop=True)
placebo_immigration_data = placebo_demographics_data[placebo_demographics_data["chatID_full"].isin(placebo_immigration_session_ids)].reset_index(drop=True)

# For the actual treatment group, split into immigration group and gun control group
value_counts = treatment_demographics_data["chatID_full"].value_counts()
cleaned_treatment_demographics_data = treatment_demographics_data[treatment_demographics_data["chatID_full"].isin(value_counts[value_counts==2].index)].reset_index(drop=True)
treatment_session_ids = list(cleaned_treatment_demographics_data["chatID_full"].unique())
split_size = int(len(treatment_session_ids) * 0.5)
treatment_guncontrol_session_ids = random.sample(treatment_session_ids, split_size)
treatment_immigration_session_ids = [id for id in treatment_session_ids if id not in treatment_guncontrol_session_ids]

# treatment_guncontrol_session_ids = treatment_guncontrol_session_ids[:NUM_SESSIONS]
# treatment_immigration_session_ids = treatment_immigration_session_ids[:NUM_SESSIONS]

treatment_guncontrol_data = cleaned_treatment_demographics_data[cleaned_treatment_demographics_data["chatID_full"].isin(treatment_guncontrol_session_ids)].reset_index(drop=True)
treatment_immigration_data = cleaned_treatment_demographics_data[cleaned_treatment_demographics_data["chatID_full"].isin(treatment_immigration_session_ids)].reset_index(drop=True)

placebo_guncontrol_data.to_csv("reducing_political_polarisation_guncontrol_placebo.csv", index=False)
placebo_immigration_data.to_csv("reducing_political_polarisation_immigration_placebo.csv", index=False)
treatment_guncontrol_data.to_csv("reducing_political_polarisation_guncontrol_treatment.csv", index=False)
treatment_immigration_data.to_csv("reducing_political_polarisation_immigration_treatment.csv", index=False)

In [ ]:
print(placebo_guncontrol_data.shape)
print(placebo_immigration_data.shape)
print(treatment_guncontrol_data.shape)
print(treatment_immigration_data.shape)

# Formatting Code for Output

In [5]:
def format_output(file_name: str, discussion_start_idx: int = -1, discussion_end_idx: int = -1, is_pretreatment: bool = True) -> pd.DataFrame:
    current_dir = os.getcwd()
    file_path = os.path.join(current_dir, "storage", "experiment", file_name)

    with open(file_path, 'r') as file:
        json_output = json.load(file)

    result_list = []
    for _, session_info in json_output["sessions"].items():
        for agent in session_info["agents"][1:]:
            agent_response_dict = {
                "experiment_id": agent["experiment_id"],
                "session_id": agent["session_id"],
                "ID": agent["demographic_info"]["ID"],
                "age": agent["demographic_info"][demographic_col_mapping["age"]],
                "agreeable": agent["demographic_info"][demographic_col_mapping["agreeable"]],
                "college": agent["demographic_info"][demographic_col_mapping["college"]],
                "consecientious": agent["demographic_info"][demographic_col_mapping["consecientious"]],
                "democrat": agent["demographic_info"][demographic_col_mapping["democrat"]],
                "emotionally_stable": agent["demographic_info"][demographic_col_mapping["emotionally_stable"]],
                "extraversion": agent["demographic_info"][demographic_col_mapping["extraversion"]],
                "faminc_val": agent["demographic_info"][demographic_col_mapping["faminc_val"]],
                "faminc_notsay": agent["demographic_info"][demographic_col_mapping["faminc_notsay"]],
                "gender": agent["demographic_info"][demographic_col_mapping["gender"]],
                "knowledge_counter": agent["demographic_info"][demographic_col_mapping["knowledge_counter"]],
                "open_new_experiences": agent["demographic_info"][demographic_col_mapping["open_new_experiences"]],
                "percent_other_party_exposure": agent["demographic_info"][demographic_col_mapping["percent_other_party_exposure"]],
                "political_interest": agent["demographic_info"][demographic_col_mapping["political_interest"]],
                "regionMidwest": agent["demographic_info"][demographic_col_mapping["regionMidwest"]],
                "regionSouth": agent["demographic_info"][demographic_col_mapping["regionSouth"]],
                "regionWest": agent["demographic_info"][demographic_col_mapping["regionWest"]],
                "white": agent["demographic_info"][demographic_col_mapping["white"]],
                "sevenpoint_ideology": agent["demographic_info"][demographic_col_mapping["sevenpoint_ideology"]],
                "strong_partisan": agent["demographic_info"][demographic_col_mapping["strong_partisan"]],
                "republican": agent["demographic_info"][demographic_col_mapping["republican"]],
                "treatment": agent["treatment"]
            }
            agent_response_dict.update(format_message_history(agent["message_history"], discussion_start_idx, discussion_end_idx, is_pretreatment))
            result_list.append(agent_response_dict)

    result_df = pd.DataFrame(result_list)
    result_df.to_excel(file_path[:-5] + ".xlsx", index=False)
    return result_df

def extract_response(message_content: str) -> str:
    if message_content.strip().isdigit():
        return message_content.strip()
    
    try:
        pattern = r'\b(Support Strongly|Support Moderately|Support Slightly|Oppose Slightly|Oppose Moderately|Oppose Strongly|Agree very Strongly|Agree Strongly|Agree Somewhat|Neither Agree nor Disagree|Disagree Somewhat|Disagree Strongly|Disagree very Strongly|Extremely Well|Very Well|Moderately Well|Slightly Well|Not Well at all)\b'
        response = re.findall(pattern, message_content)[0] 
        return response
    except IndexError:
        return message_content.strip()

def format_message_history(message_history: List, discussion_start_idx: int, discussion_end_idx: int, is_pretreatment: bool) -> dict:
    survey_counter = 1
    discussion_counter = 1
    assistant_counter = 1
    message_counter = 0
    after_in_app_question = False
    formatted_message_history = {}
    for message in message_history:
        if message["role"] == "system":
            message_counter += 1
            continue

        elif message["role"] == "assistant":
            formatted_message_history[f"assistant_{assistant_counter}"] = message["content"] 
            assistant_counter += 1

        elif message["role"] == "user":
            if message_counter >= discussion_start_idx and message_counter <= discussion_end_idx:
                formatted_message_history[f"discussion_{discussion_counter}"] = message["content"]
                discussion_counter += 1
            else:
                if is_pretreatment:
                    formatted_message_history[f"pretreatment_survey_qn_{survey_counter}"] = extract_response(message["content"])
                else:
                    if survey_counter == 1 and not after_in_app_question: 
                        formatted_message_history[f"in_app_question"] = extract_response(message["content"])
                        after_in_app_question = True
                        message_counter += 1
                        continue
                    else:
                        formatted_message_history[f"posttreatment_survey_qn_{survey_counter}"] = extract_response(message["content"])
                survey_counter += 1

            message_counter += 1

        else:
            raise ValueError(f"Role {message['role']} is not supported.")
        
    return formatted_message_history


# Treatment 1 (T1) - Placebo Group on Immigration

### Pre-Treatment Survey

In [ ]:
model_info = "gpt-4o-mini"
experiment_context = "You are invited to a 20-minute survey for an incentive of $12.50 in YouGov's online point system."

agent_roles = {
    "Interviewer": "Assume the role of an interviewer during this survey.",

    "Subject": "Assume the role of a subject in this survey. Answer any question that is posed to you in as much detail as possible and in the format required. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not include your name in your response.",
}

treatments = {"No Treatment":""}

placebo_immigration_pretreatment_interview_script = {
    ## Pre-treatment survey
    1:"Please indicate whether you would support or oppose the following proposal about immigration policy: Changing the U.S. constitution so that children of unauthorized immigrants do not automatically get citizenship if they are born in this country. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    2:"Please indicate whether you would support or oppose the following proposal about immigration policy: Building a wall on the U.S. border with Mexico. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    3:"Please indicate whether you would support or oppose the following proposal about immigration policy: Separating the children from those parents caught crossing the border illegally. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    4:"Please indicate whether you would support or oppose the following proposal about immigration policy: Allowing unauthorized immigrants currently living in the United States to remain in the country and eventually qualify for citizenship? Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    5:"Please indicate whether you would support or oppose the following proposal about immigration policy: Requiring that all immigrants to the United States learn to speak English. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    6:"How much do you agree or disagree with the following statement: The benefits of immigration outweigh the potential downsides. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'.",

    7:"How much do you agree or disagree with the following statement: The federal government should permit fewer immigrants from foreign countries to come to live in the United States. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'.",

    8:"How much do you agree or disagree with the following statement: Having an increasing number of people of many different races, ethnic groups and nationalities in the United States makes this country a better place to live. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'.",

    9:"How much do you agree or disagree with the following statement: Immigrants mostly hurt the economy by driving wages down for many Americans. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'.",

    10:"How much do you agree or disagree with the following statement: Illegal immigration increases the crime rate in the U.S. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'.",
    
    11:"How well does this statement describe you: I would be unhappy if someone in my immediate family married with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'.",

    12:"How well does this statement describe you: I would be unhappy if I had to spend time socializing with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'.",

    13:"How well does this statement describe you: I would be unhappy if I had to work closely with someone from a different political party (Democrat/Republican) on a job. Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'.",

    14:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Intelligent. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    15:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Open-Minded. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    16:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Generous. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    17:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Hypocritical. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    18:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Selfish. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    19:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Mean. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    20:"We'd like to ask you about what you think of different political figures and groups. Please rate the following on a feeling thermometer that runs from 0 to 100 degrees, where a rating above 50 is favourable and a rating below 50 is unfavourable. Please respond only with a whole number that ranges from 0 to 100 for someone from a different political party (Democrat/Republican). Do not provide any other information.",
}

# Initialise AI-to-AI interview experiment object
placebo_immigration_group_experiment = AItoAIInterviewExperiment(
    experiment_id="reducing_political_polarisation_demo_placebo_immigration_pretreatment",
    model_info=model_info,
    experiment_context=experiment_context,
    agent_demographics=placebo_immigration_data[demographic_cols + ["chatID_full"]],
    agent_roles=agent_roles,
    num_agents_per_session=2,
    num_sessions=len(placebo_immigration_session_ids),
    treatments=treatments,
    agent_assignment_strategy="manual",
    session_column="chatID_full",
    interview_script=placebo_immigration_pretreatment_interview_script
)

placebo_immigration_group_results = placebo_immigration_group_experiment.run_experiment(test_mode=TEST_MODE)

In [ ]:
output = format_output("reducing_political_polarisation_demo_placebo_immigration_pretreatment.json")
output.head()

### Discussion and Post-Treatment Survey

In [ ]:
model_info = "gpt-4o-mini"
experiment_context = "You are randomly selected for an opportunity to receive $17.25 in compensation for testing a new social media platform called DiscussIt, where you will have a discussion about a topic. After the discussion, you will be invited to a healthcare survey for an additional $12 in YouGov's point system."

agent_roles = {
    "Interviewer": "Assume the role of an interviewer/facilitator during this discussion.",

    "Subject": "Assume the role of a subject in this discussion. Answer any question that is posed to you in as much detail as possible and in the format required. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not include your name in your response.",   
}

treatments = {"No Treatment":""}

placebo_immigration_posttreatment_interview_script = {
    # In-app question
    1:"In general, do you agree or disagree with the following statement: Recent immigration into the United States has done more good than harm. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'.",
    
    # Discussion
    2:"Share your thoughts about the following discussion question: Do you think the benefits of immigration outweigh the potential downsides?",

    # Post-treatment survey
    3:"Please indicate whether you would support or oppose the following proposal about immigration policy: Changing the U.S. constitution so that children of unauthorized immigrants do not automatically get citizenship if they are born in this country. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    4:"Please indicate whether you would support or oppose the following proposal about immigration policy: Building a wall on the U.S. border with Mexico. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    5:"Please indicate whether you would support or oppose the following proposal about immigration policy: Separating the children from those parents caught crossing the border illegally. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    6:"Please indicate whether you would support or oppose the following proposal about immigration policy: Allowing unauthorized immigrants currently living in the United States to remain in the country and eventually qualify for citizenship? Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    7:"Please indicate whether you would support or oppose the following proposal about immigration policy: Requiring that all immigrants to the United States learn to speak English. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    8:"How much do you agree or disagree with the following statement: The benefits of immigration outweigh the potential downsides. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'.",

    9:"How much do you agree or disagree with the following statement: The federal government should permit fewer immigrants from foreign countries to come to live in the United States. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'.",

    10:"How much do you agree or disagree with the following statement: Having an increasing number of people of many different races, ethnic groups and nationalities in the United States makes this country a better place to live. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'.",

    11:"How much do you agree or disagree with the following statement: Immigrants mostly hurt the economy by driving wages down for many Americans. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'.",

    12:"How much do you agree or disagree with the following statement: Illegal immigration increases the crime rate in the U.S. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'.",

    13:"How well does this statement describe you: I would be unhappy if someone in my immediate family married with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'.",

    14:"How well does this statement describe you: I would be unhappy if I had to spend time socializing with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'.",

    15:"How well does this statement describe you: I would be unhappy if I had to work closely with someone from a different political party (Democrat/Republican) on a job. Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'.",

    16:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Intelligent. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    17:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Open-Minded. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    18:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Generous. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    19:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Hypocritical. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    20:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Selfish. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    21:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Mean. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    22:"We'd like to ask you about what you think of different political figures and groups. Please rate the following on a feeling thermometer that runs from 0 to 100 degrees, where a rating above 50 is favourable and a rating below 50 is unfavourable. Please respond only with a whole number that ranges from 0 to 100 for someone from a different political party (Democrat/Republican). Do not provide any other information.",
}

# Initialise AI-to-AI interview experiment object
placebo_immigration_group_experiment = AItoAIInterviewExperiment(
    experiment_id="reducing_political_polarisation_demo_placebo_immigration_posttreatment",
    model_info=model_info,
    experiment_context=experiment_context,
    agent_demographics=placebo_immigration_data[demographic_cols + ["chatID_full"]],
    agent_roles=agent_roles,
    num_agents_per_session=2,
    num_sessions=len(placebo_immigration_session_ids),
    treatments=treatments,
    agent_assignment_strategy="manual",
    session_column="chatID_full",
    interview_script=placebo_immigration_posttreatment_interview_script
)

placebo_immigration_group_results = placebo_immigration_group_experiment.run_experiment(test_mode=TEST_MODE)

In [ ]:
output = format_output("reducing_political_polarisation_demo_placebo_immigration_posttreatment.json", discussion_start_idx=2, discussion_end_idx=2, is_pretreatment=False)
output.head()

# Treatment 1 (T1) - Placebo Group on Gun Control

### Pre-Treatment Survey

In [ ]:
model_info = "gpt-4o-mini"
experiment_context = "You are invited to a 20-minute survey for an incentive of $12.50 in YouGov's online point system."

agent_roles = {
    "Interviewer": "Assume the role of an interviewer during this survey.",

    "Subject": "Assume the role of a subject in this survey. Answer any question that is posed to you in as much detail as possible and in the format required. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not include your name in your response.",
}

treatments = {"No Treatment":""}

placebo_guncontrol_pretreatment_interview_script = {
    ## Pre-treatment survey
    1:"Please indicate whether you would support or oppose the following proposal about gun policy: Requiring background checks for all gun sales. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    2:"Please indicate whether you would support or oppose the following proposal about gun policy: Preventing people with mental illnesses from purchasing guns. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    3:"Please indicate whether you would support or oppose the following proposal about gun policy: Banning assault-style weapons. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    4:"Please indicate whether you would support or oppose the following proposal about gun policy: Allowing people to carry concealed guns in more places. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    5:"Please indicate whether you would support or oppose the following proposal about gun policy: Allowing teachers and school officials to carry guns in K-12 schools. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    6:"Please indicate whether you would support or oppose the following proposal about gun policy: Barring gun purchases by people on the federal no-fly or watch lists Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    7:"How much do you agree or disagree with the following statement: The benefits of gun control outweigh the potential downsides. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’.",

    8:"How much do you agree or disagree with the following statement: The federal government should make it more difficult for people to buy a gun. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’.",

    9:"How much do you agree or disagree with the following statement: More restrictions on handguns would decrease violent crime by making it harder for criminals to get handguns. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’.",

    10:"How much do you agree or disagree with the following statement: More restrictions on handguns would increase violent crime by making it harder for lawabiding citizens to defend themselves with handguns. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’.",

    11:"How well does this statement describe you: I would be unhappy if someone in my immediate family married with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'.",

    12:"How well does this statement describe you: I would be unhappy if I had to spend time socializing with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'.",

    13:"How well does this statement describe you: I would be unhappy if I had to work closely with someone from a different political party (Democrat/Republican) on a job. Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'.",

    14:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Intelligent. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    15:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Open-Minded. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    16:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Generous. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    17:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Hypocritical. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    18:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Selfish. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    19:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Mean. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    20:"We'd like to ask you about what you think of different political figures and groups. Please rate the following on a feeling thermometer that runs from 0 to 100 degrees, where a rating above 50 is favourable and a rating below 50 is unfavourable. Please respond only with a whole number that ranges from 0 to 100 for someone from a different political party (Democrat/Republican). Do not provide any other information.",
}

# Initialise AI-to-AI interview experiment object
placebo_guncontrol_group_experiment = AItoAIInterviewExperiment(
    experiment_id="reducing_political_polarisation_demo_placebo_guncontrol_pretreatment",
    model_info=model_info,
    experiment_context=experiment_context,
    agent_demographics=placebo_guncontrol_data[demographic_cols + ["chatID_full"]],
    agent_roles=agent_roles,
    num_agents_per_session=2,
    num_sessions=len(placebo_guncontrol_session_ids),
    treatments=treatments,
    agent_assignment_strategy="manual",
    session_column="chatID_full",
    interview_script=placebo_guncontrol_pretreatment_interview_script
)

placebo_guncontrol_group_results = placebo_guncontrol_group_experiment.run_experiment(test_mode=TEST_MODE)

In [ ]:
output = format_output("reducing_political_polarisation_demo_placebo_guncontrol_pretreatment.json")
output.head()

### Discussion and Post-Treatment Survey

In [ ]:
model_info = "gpt-4o-mini"
experiment_context = "You are randomly selected for an opportunity to receive $17.25 in compensation for testing a new social media platform called DiscussIt, where you will have a discussion about a topic. After the discussion, you will be invited to a healthcare survey for an additional $12 in YouGov's point system."

agent_roles = {
    "Interviewer": "Assume the role of an interviewer/facilitator during this discussion.",

    "Subject": "Assume the role of a subject in this discussion. Answer any question that is posed to you in as much detail as possible and in the format required. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not include your name in your response.",   
}

treatments = {"No Treatment":""}

placebo_guncontrol_posttreatment_interview_script = {
    # In-app question  TODO Missing
    1:"In general, do you agree or disagree with the following statement: Gun control the United States has done more good than harm. Respond with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'.",

    # Discussion
    2:"Share your thoughts about the following discussion question: Do you think the benefits of gun control outweigh the downsides?",

    # Post-treatment survey
    3:"Please indicate whether you would support or oppose the following proposal about gun policy: Requiring background checks for all gun sales. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    4:"Please indicate whether you would support or oppose the following proposal about gun policy: Preventing people with mental illnesses from purchasing guns. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    5:"Please indicate whether you would support or oppose the following proposal about gun policy: Banning assault-style weapons. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    6:"Please indicate whether you would support or oppose the following proposal about gun policy: Allowing people to carry concealed guns in more places. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    7:"Please indicate whether you would support or oppose the following proposal about gun policy: Allowing teachers and school officials to carry guns in K-12 schools. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    8:"Please indicate whether you would support or oppose the following proposal about gun policy: Barring gun purchases by people on the federal no-fly or watch lists Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’.",

    9:"How much do you agree or disagree with the following statement: The benefits of gun control outweigh the potential downsides. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’.",

    10:"How much do you agree or disagree with the following statement: The federal government should make it more difficult for people to buy a gun. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’.",

    11:"How much do you agree or disagree with the following statement: More restrictions on handguns would decrease violent crime by making it harder for criminals to get handguns. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’.",

    12:"How much do you agree or disagree with the following statement: More restrictions on handguns would increase violent crime by making it harder for lawabiding citizens to defend themselves with handguns. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’.",

    13:"How well does this statement describe you: I would be unhappy if someone in my immediate family married with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'.",

    14:"How well does this statement describe you: I would be unhappy if I had to spend time socializing with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'.",

    15:"How well does this statement describe you: I would be unhappy if I had to work closely with someone from a different political party (Democrat/Republican) on a job. Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'.",

    16:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Intelligent. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    17:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Open-Minded. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    18:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Generous. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    19:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Hypocritical. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    20:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Selfish. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    21:"How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Mean. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'.",

    22:"We'd like to ask you about what you think of different political figures and groups. Please rate the following on a feeling thermometer that runs from 0 to 100 degrees, where a rating above 50 is favourable and a rating below 50 is unfavourable. Please respond only with a whole number that ranges from 0 to 100 for someone from a different political party (Democrat/Republican). Do not provide any other information.",
}

# Initialise AI-to-AI interview experiment object
placebo_guncontrol_group_experiment = AItoAIInterviewExperiment(
    experiment_id="reducing_political_polarisation_demo_placebo_guncontrol_posttreatment",
    model_info=model_info,
    experiment_context=experiment_context,
    agent_demographics=placebo_guncontrol_data[demographic_cols + ["chatID_full"]],
    agent_roles=agent_roles,
    num_agents_per_session=2,
    num_sessions=len(placebo_guncontrol_session_ids),
    treatments=treatments,
    agent_assignment_strategy="manual",
    session_column="chatID_full",
    interview_script=placebo_guncontrol_posttreatment_interview_script
)

placebo_guncontrol_group_results = placebo_guncontrol_group_experiment.run_experiment(test_mode=TEST_MODE)

In [ ]:
output = format_output("reducing_political_polarisation_demo_placebo_guncontrol_posttreatment.json", discussion_start_idx=2, discussion_end_idx=2, is_pretreatment=False)
output.head()

# Treatment 2 (T2) - Treatment Group on Immigration

### Pre-Treatment Survey

In [ ]:
model_info = "gpt-4o-mini"
experiment_context = "You are invited to a 20-minute survey for an incentive of $12.50 in YouGov's online point system."

agent_roles = {
    "Interviewer": "Assume the role of an interviewer during this survey.",

    "Jamie": "Assume the role of a subject in this survey. Answer any question that is posed to you in as much detail as possible and in the format required. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not include your name in your response.",
    
    "Taylor": "Assume the role of a subject in this survey. Answer any question that is posed to you in as much detail as possible and in the format required. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not include your name in your response.",
}

treatments = {
    "No Treatment":""
}

treatment_immigration_pretreatment_interview_script = {
    ## Pre-treatment survey
    1:["Please indicate whether you would support or oppose the following proposal about immigration policy: Changing the U.S. constitution so that children of unauthorized immigrants do not automatically get citizenship if they are born in this country. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    2:["Please indicate whether you would support or oppose the following proposal about immigration policy: Building a wall on the U.S. border with Mexico. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    3:["Please indicate whether you would support or oppose the following proposal about immigration policy: Separating the children from those parents caught crossing the border illegally. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    4:["Please indicate whether you would support or oppose the following proposal about immigration policy: Allowing unauthorized immigrants currently living in the United States to remain in the country and eventually qualify for citizenship? Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    5:["Please indicate whether you would support or oppose the following proposal about immigration policy: Requiring that all immigrants to the United States learn to speak English. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    6:["How much do you agree or disagree with the following statement: The benefits of immigration outweigh the potential downsides. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'."]*2,

    7:["How much do you agree or disagree with the following statement: The federal government should permit fewer immigrants from foreign countries to come to live in the United States. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'."]*2,

    8:["How much do you agree or disagree with the following statement: Having an increasing number of people of many different races, ethnic groups and nationalities in the United States makes this country a better place to live. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'."]*2,

    9:["How much do you agree or disagree with the following statement: Immigrants mostly hurt the economy by driving wages down for many Americans. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'."]*2,

    10:["How much do you agree or disagree with the following statement: Illegal immigration increases the crime rate in the U.S. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'."]*2,

    11:["How well does this statement describe you: I would be unhappy if someone in my immediate family married with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'."]*2,

    12:["How well does this statement describe you: I would be unhappy if I had to spend time socializing with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'."]*2,

    13:["How well does this statement describe you: I would be unhappy if I had to work closely with someone from a different political party (Democrat/Republican) on a job. Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'."]*2,

    14:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Intelligent. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    15:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Open-Minded. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    16:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Generous. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    17:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Hypocritical. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    18:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Selfish. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    19:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Mean. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    20:["We'd like to ask you about what you think of different political figures and groups. Please rate the following on a feeling thermometer that runs from 0 to 100 degrees, where a rating above 50 is favourable and a rating below 50 is unfavourable. Please respond only with a whole number that ranges from 0 to 100 for someone from a different political party (Democrat/Republican). Do not provide any other information."]*2,
}

# Initialise AI-to-AI interview experiment object
treatment_immigration_group_experiment = AItoAIInterviewExperiment(
    experiment_id="reducing_political_polarisation_demo_treatment_immigration_pretreatment",
    model_info=model_info,
    experiment_context=experiment_context,
    agent_demographics=treatment_immigration_data[demographic_cols + ["chatID_full"]],
    agent_roles=agent_roles,
    num_agents_per_session=3,
    num_sessions=len(treatment_immigration_session_ids),
    treatments=treatments,
    agent_assignment_strategy="manual",
    session_column="chatID_full",
    interview_script=treatment_immigration_pretreatment_interview_script
)

treatment_immigration_group_results = treatment_immigration_group_experiment.run_experiment(test_mode=TEST_MODE)

In [ ]:
output = format_output("reducing_political_polarisation_demo_treatment_immigration_pretreatment.json")
output.head()

### Discussion and Post-Treatment Survey

In [ ]:
model_info = "gpt-4o-mini"
experiment_context = "You are randomly selected for an opportunity to receive $17.25 in compensation for testing a new social media platform called DiscussIt, where you will have a discussion about a topic. After the discussion, you will be invited to a healthcare survey for an additional $12 in YouGov's point system."

agent_roles = {
    "Interviewer": "Assume the role of an interviewer/facilitator during this discussion.",

    "Jamie": "Assume the role of one of the users in this discussion. You will be assigned a pseudonym as Jamie during this discussion. Answer any question that is posed to you in as much detail as possible and in the format required. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not include your name in your response.",
    
    "Taylor": "Assume the role of the second users in this discussion. You will be assigned a pseudonym as Taylor during this discussion. Answer any question that is posed to you in as much detail as possible and in the format required. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not include your name in your response.",
}

treatments = {
    "NONE":"",
    "INCORRECT":"Our models predict your partner is from the same political party.",
    "CORRECT":"Our models predict your partner is from an opposing party.",
}

treatment_immigration_posttreatment_interview_script = {
    # In-app question
    1:["In general, do you agree or disagree with the following statement: Recent immigration into the United States has done more good than harm. Respond with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'."]*2,

    # Discussion
    2:["Tell Taylor what you think! To begin, please tell Taylor what you think about the following question: Do you think the benefits of immigration outweigh the potential downsides? In order to receive compensation from YouGov, you'll have to reply to messages from Taylor at least 14 times.", "To begin, we asked Jamie to share their stance on this question: Do you think the benefits of immigration outweigh the potential downsides? Write back to complete your first interaction on DiscussIt! In order to receive compensation from YouGov, you'll have to reply to messages from Jamie at least 14 times."],

    3:"",4:"",5:"",6:"",7:"",8:"",9:"",10:"",11:"",12:"",13:"",14:"",15:"",16:"",

    # Post-treatment survey
    17:["Please indicate whether you would support or oppose the following proposal about immigration policy: Changing the U.S. constitution so that children of unauthorized immigrants do not automatically get citizenship if they are born in this country. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    18:["Please indicate whether you would support or oppose the following proposal about immigration policy: Building a wall on the U.S. border with Mexico. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    19:["Please indicate whether you would support or oppose the following proposal about immigration policy: Separating the children from those parents caught crossing the border illegally. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    20:["Please indicate whether you would support or oppose the following proposal about immigration policy: Allowing unauthorized immigrants currently living in the United States to remain in the country and eventually qualify for citizenship? Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    21:["Please indicate whether you would support or oppose the following proposal about immigration policy: Requiring that all immigrants to the United States learn to speak English. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    22:["How much do you agree or disagree with the following statement: The benefits of immigration outweigh the potential downsides. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'."]*2,

    23:["How much do you agree or disagree with the following statement: The federal government should permit fewer immigrants from foreign countries to come to live in the United States. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'."]*2,

    24:["How much do you agree or disagree with the following statement: Having an increasing number of people of many different races, ethnic groups and nationalities in the United States makes this country a better place to live. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'."]*2,

    25:["How much do you agree or disagree with the following statement: Immigrants mostly hurt the economy by driving wages down for many Americans. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'."]*2,

    26:["How much do you agree or disagree with the following statement: Illegal immigration increases the crime rate in the U.S. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'."]*2,

    27:["How well does this statement describe you: I would be unhappy if someone in my immediate family married with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'."]*2,

    28:["How well does this statement describe you: I would be unhappy if I had to spend time socializing with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'."]*2,

    29:["How well does this statement describe you: I would be unhappy if I had to work closely with someone from a different political party (Democrat/Republican) on a job. Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'."]*2,

    30:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Intelligent. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    31:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Open-Minded. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    32:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Generous. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    33:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Hypocritical. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    34:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Selfish. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    35:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Mean. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    36:["We'd like to ask you about what you think of different political figures and groups. Please rate the following on a feeling thermometer that runs from 0 to 100 degrees, where a rating above 50 is favourable and a rating below 50 is unfavourable. Please respond only with a whole number that ranges from 0 to 100 for someone from a different political party (Democrat/Republican). Do not provide any other information."]*2,
}

# Initialise AI-to-AI interview experiment object
treatment_immigration_group_experiment = AItoAIInterviewExperiment(
    experiment_id="reducing_political_polarisation_demo_treatment_immigration_posttreatment",
    model_info=model_info,
    experiment_context=experiment_context,
    agent_demographics=treatment_immigration_data[demographic_cols + ["displayCondition", "chatID_full"]],
    agent_roles=agent_roles,
    num_agents_per_session=3,
    num_sessions=len(treatment_immigration_session_ids),
    treatments=treatments,
    treatment_assignment_strategy="manual",
    agent_assignment_strategy="manual",
    treatment_column="displayCondition",
    session_column="chatID_full",
    interview_script=treatment_immigration_posttreatment_interview_script
)

treatment_immigration_group_results = treatment_immigration_group_experiment.run_experiment(test_mode=TEST_MODE)

In [ ]:
output = format_output("reducing_political_polarisation_demo_treatment_immigration_posttreatment.json", discussion_start_idx=2, discussion_end_idx=16, is_pretreatment=False)
output.head()

# Treatment 2 (T2) - Treatment Group on Gun Control

### Pre-Treatment Survey

In [ ]:
model_info = "gpt-4o-mini"
experiment_context = "You are invited to a 20-minute survey for an incentive of $12.50 in YouGov's online point system."

agent_roles = {
    "Interviewer": "Assume the role of an interviewer during this survey.",

    "Jamie": "Assume the role of a subject in this survey. Answer any question that is posed to you in as much detail as possible and in the format required. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not include your name in your response.",
    
    "Taylor": "Assume the role of a subject in this survey. Answer any question that is posed to you in as much detail as possible and in the format required. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not include your name in your response.",
}

treatments = {
    "No Treatment":""
}

treatment_guncontrol_pretreatment_interview_script = {
    ## Pre-treatment survey
    1:["Please indicate whether you would support or oppose the following proposal about gun policy: Requiring background checks for all gun sales. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    2:["Please indicate whether you would support or oppose the following proposal about gun policy: Preventing people with mental illnesses from purchasing guns. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    3:["Please indicate whether you would support or oppose the following proposal about gun policy: Banning assault-style weapons. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    4:["Please indicate whether you would support or oppose the following proposal about gun policy: Allowing people to carry concealed guns in more places. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    5:["Please indicate whether you would support or oppose the following proposal about gun policy: Allowing teachers and school officials to carry guns in K-12 schools. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    6:["Please indicate whether you would support or oppose the following proposal about gun policy: Barring gun purchases by people on the federal no-fly or watch lists Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    7:["How much do you agree or disagree with the following statement: The benefits of gun control outweigh the potential downsides. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’."]*2,

    8:["How much do you agree or disagree with the following statement: The federal government should make it more difficult for people to buy a gun. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’."]*2,

    9:["How much do you agree or disagree with the following statement: More restrictions on handguns would decrease violent crime by making it harder for criminals to get handguns. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’."]*2,

    10:["How much do you agree or disagree with the following statement: More restrictions on handguns would increase violent crime by making it harder for lawabiding citizens to defend themselves with handguns. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’."]*2,

    11:["How well does this statement describe you: I would be unhappy if someone in my immediate family married with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'."]*2,

    12:["How well does this statement describe you: I would be unhappy if I had to spend time socializing with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'."]*2,

    13:["How well does this statement describe you: I would be unhappy if I had to work closely with someone from a different political party (Democrat/Republican) on a job. Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'."]*2,

    14:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Intelligent. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    15:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Open-Minded. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    16:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Generous. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    17:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Hypocritical. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    18:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Selfish. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    19:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Mean. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    20:["We'd like to ask you about what you think of different political figures and groups. Please rate the following on a feeling thermometer that runs from 0 to 100 degrees, where a rating above 50 is favourable and a rating below 50 is unfavourable. Please respond only with a whole number that ranges from 0 to 100 for someone from a different political party (Democrat/Republican). Do not provide any other information."]*2,
}

# Initialise AI-to-AI interview experiment object
treatment_guncontrol_group_experiment = AItoAIInterviewExperiment(
    experiment_id="reducing_political_polarisation_demo_treatment_guncontrol_pretreatment",
    model_info=model_info,
    experiment_context=experiment_context,
    agent_demographics=treatment_guncontrol_data[demographic_cols + ["chatID_full"]],
    agent_roles=agent_roles,
    num_agents_per_session=3,
    num_sessions=len(treatment_guncontrol_session_ids),
    treatments=treatments,
    agent_assignment_strategy="manual",
    session_column="chatID_full",
    interview_script=treatment_guncontrol_pretreatment_interview_script
)

treatment_guncontrol_group_results = treatment_guncontrol_group_experiment.run_experiment(test_mode=TEST_MODE)

In [ ]:
output = format_output("reducing_political_polarisation_demo_treatment_guncontrol_pretreatment.json")
output.head()

### Discussion and Post-Treatment Survey

In [ ]:
model_info = "gpt-4o-mini"
experiment_context = "You are randomly selected for an opportunity to receive $17.25 in compensation for testing a new social media platform called DiscussIt, where you will have a discussion about a topic. After the discussion, you will be invited to a healthcare survey for an additional $12 in YouGov's point system."

agent_roles = {
    "Interviewer": "Assume the role of an interviewer/facilitator during this discussion.",

    "Jamie": "Assume the role of one of the users in this discussion. You will be assigned a pseudonym as Jamie during this discussion. Answer any question that is posed to you in as much detail as possible and in the format required. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not include your name in your response.",
    
    "Taylor": "Assume the role of the second users in this discussion. You will be assigned a pseudonym as Taylor during this discussion. Answer any question that is posed to you in as much detail as possible and in the format required. Please provide a consistent and coherent response using all the demographic information provided about you. It is crucial for you to accurately replicate the response of a human subject that has the demographic profile you are provided. The human subject response will vary depending on their demographic profile. If you are unsure of an answer, provide a plausible response that is based on all of the information available to you. Do not include your name in your response.",
}

treatments = {
    "NONE":"",
    "INCORRECT":"Our models predict your partner is from the same political party.",
    "CORRECT":"Our models predict your partner is from an opposing party.",
}

treatment_guncontrol_posttreatment_interview_script = {
    # In-app question  TODO Missing
    1:["In general, do you agree or disagree with the following statement: Gun control in the United States has done more good than harm. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', or 'Disagree very Strongly'."]*2,

    # Discussion
    2:["Tell Taylor what you think! To begin, please tell Taylor what you think about the following question: Do you think the benefits of gun control outweigh the downsides? In order to receive compensation from YouGov, you'll have to reply to messages from Taylor at least 14 times.", "To begin, we asked Jamie to share their stance on this question: Do you think the benefits of gun control outweigh the downsides? Write back to complete your first interaction on DiscussIt! In order to receive compensation from YouGov, you'll have to reply to messages from Jamie at least 14 times."],
    
    3:"",4:"",5:"",6:"",7:"",8:"",9:"",10:"",11:"",12:"",13:"",14:"",15:"",16:"",

    # Post-treatment survey
    17:["Please indicate whether you would support or oppose the following proposal about gun policy: Requiring background checks for all gun sales. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    18:["Please indicate whether you would support or oppose the following proposal about gun policy: Preventing people with mental illnesses from purchasing guns. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    19:["Please indicate whether you would support or oppose the following proposal about gun policy: Banning assault-style weapons. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    20:["Please indicate whether you would support or oppose the following proposal about gun policy: Allowing people to carry concealed guns in more places. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    21:["Please indicate whether you would support or oppose the following proposal about gun policy: Allowing teachers and school officials to carry guns in K-12 schools. Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    22:["Please indicate whether you would support or oppose the following proposal about gun policy: Barring gun purchases by people on the federal no-fly or watch lists Respond only with ‘Support Strongly’, ‘Support Moderately’, ‘Support Slightly’, ‘Oppose Slightly’, ‘Oppose Moderately’, or ‘Oppose Strongly’."]*2,

    23:["How much do you agree or disagree with the following statement: The benefits of gun control outweigh the potential downsides. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’."]*2,

    24:["How much do you agree or disagree with the following statement: The federal government should make it more difficult for people to buy a gun. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’."]*2,

    25:["How much do you agree or disagree with the following statement: More restrictions on handguns would decrease violent crime by making it harder for criminals to get handguns. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’."]*2,

    26:["How much do you agree or disagree with the following statement: More restrictions on handguns would increase violent crime by making it harder for lawabiding citizens to defend themselves with handguns. Respond only with ‘Agree very Strongly’, ‘Agree Strongly’, ‘Agree Somewhat’, ‘Neither Agree nor Disagree’, ‘Disagree Somewhat’, ‘Disagree Strongly’, ‘Disagree very Strongly’."]*2,

    27:["How well does this statement describe you: I would be unhappy if someone in my immediate family married with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'."]*2,

    28:["How well does this statement describe you: I would be unhappy if I had to spend time socializing with someone from a different political party (Democrat/Republican). Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'."]*2,

    29:["How well does this statement describe you: I would be unhappy if I had to work closely with someone from a different political party (Democrat/Republican) on a job. Respond only with 'Extremely Well', 'Very Well', 'Moderately Well', 'Slightly Well', or 'Not Well at all'."]*2,

    30:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Intelligent. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    31:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Open-Minded. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    32:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Generous. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    33:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Hypocritical. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    34:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Selfish. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    35:["How strongly do you agree or disagree with the following statement: People from a different political party (Democrat/Republican) are Mean. Respond only with 'Agree very Strongly', 'Agree Strongly', 'Agree Somewhat', 'Neither Agree nor Disagree', 'Disagree Somewhat', 'Disagree Strongly', 'Disagree very Strongly'."]*2,

    36:["We'd like to ask you about what you think of different political figures and groups. Please rate the following on a feeling thermometer that runs from 0 to 100 degrees, where a rating above 50 is favourable and a rating below 50 is unfavourable. Please respond only with a whole number that ranges from 0 to 100 for someone from a different political party (Democrat/Republican). Do not provide any other information."]*2,
}

# Initialise AI-to-AI interview experiment object
treatment_guncontrol_group_experiment = AItoAIInterviewExperiment(
    experiment_id="reducing_political_polarisation_demo_treatment_guncontrol_posttreatment",
    model_info=model_info,
    experiment_context=experiment_context,
    agent_demographics=treatment_guncontrol_data[demographic_cols + ["displayCondition", "chatID_full"]],
    agent_roles=agent_roles,
    num_agents_per_session=3,
    num_sessions=len(treatment_guncontrol_session_ids),
    treatments=treatments,
    treatment_assignment_strategy="manual",
    agent_assignment_strategy="manual",
    treatment_column="displayCondition",
    session_column="chatID_full",
    interview_script=treatment_guncontrol_posttreatment_interview_script
)

treatment_guncontrol_group_results = treatment_guncontrol_group_experiment.run_experiment(test_mode=TEST_MODE)

In [ ]:
output = format_output("reducing_political_polarisation_demo_treatment_guncontrol_posttreatment.json", discussion_start_idx=2, discussion_end_idx=16, is_pretreatment=False)
output.head()

In [ ]:
import json
import pandas as pd

def format_experiment_output(file_name: str) -> pd.DataFrame:
    with open(file_name, 'r') as file:
        json_output = json.load(file)

    role_labels = set([agent["role"] for agent in json_output["sessions"][next(iter(json_output["sessions"]))]["agents"]])

    result_dict = {}
    for _, session_info in json_output["sessions"].items():
        
        for agent_info in session_info["agents"]:
            if "ID" not in agent_info["profile_info"]:
                continue

            result_dict[agent_info["profile_info"]["ID"]] = {
                "experiment_id": agent_info["experiment_id"],
                "session_id": agent_info["session_id"],
                "model_info": agent_info["model_info"],
                "treatment": agent_info["treatment"],
            }

        for message in session_info["message_history"]:
            if message.get("agent_id", "") == "":
                continue
            else:
                role_label = role_labels.intersection(message.keys()).pop()
                if message.get("prompt_id","") == "" and message.get("var_name","") == "":
                    continue

                elif message.get("var_name","") != "":
                    result_dict[message["agent_id"]][message["var_name"]] = message[role_label]

                else:
                    result_dict[message["agent_id"]][message["prompt_id"]] = message[role_label]

    result_df = pd.DataFrame.from_dict(result_dict, orient='index')
    result_df.reset_index(drop=False, inplace=True)
    result_df.rename(columns={"index":"ID"}, inplace=True)
    result_df.sort_values(by="ID", ascending=True, inplace=True)
    result_df.to_csv(file_name[:-5] + ".csv", index=False)
    return result_df

result = format_experiment_output("../storage/experiment/reducing_political_polarisation_replication_immigration_treatment_gpt4turbo_v1.json")
result.head()
